<a href="https://colab.research.google.com/github/wjyChina/Template-For-Deep-Learning/blob/master/CNN_multi_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import numpy as np  #linear algebra
import pandas as pd #Only CSV IO
import os
import re
from torch.utils import data #dataloader of batch
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from torch.autograd import Variable #We can ask require grad or not
from torch.optim.optimizer import Optimizer
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import gc
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from keras.utils import np_utils

In [0]:
### Mount Google drive to gain access to data in it ###
from google.colab import drive
drive.mount('/content/drive')

In [0]:
### Change here ###
### Mount the folder of your data. You can use !ls to check all the folders in your current path ###
os.chdir("drive/My Drive/Colab Notebooks/") 
!ls

In [0]:
### Change here ###
### Read in your data ###
data=pd.read_csv('')

In [0]:
### Change here ###
### Split your data by data_y. First you need to split the original data into input and target ###
data_x=
data_y=
x_train,x_test,y_train,y_test = train_test_split(data_x,data_y,test_size=0.25,random_state=0)

In [0]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [0]:
### Change here ###
### Hyperparameters for training ###
seed = 1
learning_rate = 0.001
batch_size = 1024
n_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
### Change here ###
### CNN_2D ###
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()

        self.conv1 =  nn.Conv2d(1, 32,    kernel_size=3, stride=(1,1), padding=2) ## 28+2*2-3+1=30
        
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=(2,2),padding=0)  ## (30 -2)/2+1=15
        
        self.conv2 = nn.Conv2d(32,128, kernel_size=5,stride=(1,1),padding=2) ## (15+2*2-5)+1=15
        
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=(2,2),padding=0) ## (15-2)/2+1=7
        
        self.conv3 = nn.Conv2d(128,64,kernel_size=3,stride=(1,1),padding=1) ## (7+2-3)+1=7
        
        self.pool3 = nn.MaxPool2d(kernel_size=2,stride=(2,2),padding=0) ## (7-2)/2+1=3
                                                                        ## 3*3*64
        self.linear1 = nn.Linear(3*3*64,1024)
        
        self.linear2 = nn.Linear(1024,256)
        
        self.linear3 = nn.Linear(256, num_classes)  
        

    def forward(self, x):
        
        out = self.conv1(x)
        out = F.relu(out)
        out = self.pool1(out)
        
        out = self.conv2(out)
        out = F.relu(out)
        out = self.pool2(out)
        
        out = self.conv3(out)
        out = F.relu(out)
        out = self.pool3(out)
        
        out=out.view(-1,3*3*64)
        out = self.linear1(out)
        out = F.relu(out)
        
        out = self.linear2(out)
        out = F.relu(out)
        
        out = self.linear3(out)
        
        logits = out
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [0]:
### CNN 1D with concatenating and embedding. Binary ###
class CNN_1D(nn.Module):
    def __init__(self):
        super(NeuralNet2, self).__init__()
        
        
        self.embedding = nn.Embedding(embedding_size,max_word)    
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix_1, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.15)
         
        self.pool = nn.MaxPool1d(4)
        
        self.branch1 = nn.Conv1d(300,128,3)   #Conv1D takes batch*channel*width     
        self.branch2 = nn.Conv1d(300,128,4)
        self.branch3 = nn.Conv1d(300,128,5)
        
        self.bn_branch = nn.BatchNorm1d(128)
        self.bn_2 = nn.BatchNorm1d(64)
        
        self.conv_1=nn.Conv1d(128,64,5)
        self.linear=nn.Linear(576,256)
        self.linear2=nn.Linear(256,1)
      
    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = self.embedding_dropout(h_embedding)
        h_embedding = torch.transpose(h_embedding,1,2) 
        
        branch1=self.branch1(h_embedding)
        branch1=self.bn_branch(branch1)
        branch1=F.relu(branch1)
        branch1 = F.dropout(branch1,p=0.2,training=self.training)
        branch1=self.pool(branch1)
        branch2=self.branch2(h_embedding)
        branch2=self.bn_branch(branch2)
        branch2=F.relu(branch2)
        branch2 = F.dropout(branch2,p=0.2,training=self.training)
        branch2=self.pool(branch2)
        branch3=self.branch3(h_embedding)
        branch3=self.bn_branch(branch3)
        branch3=F.relu(branch3)
        branch3 = F.dropout(branch3,p=0.2,training=self.training)
        branch3=self.pool(branch3)
        cat=torch.cat([branch1,branch2,branch3],2)
        out=self.conv_1(cat)
        out=self.bn_2(out)
        out=F.relu(out)
        out=F.dropout(out,p=0.2,training=self.training)
        out=self.pool(out)
        out=out.view(-1,576)
        out=self.linear(out)
        out=F.relu(out)
        out=self.linear2(out)
        return out

In [0]:
### Change here if you are going to use loss function other than Cross Entropy ###
def compute_epoch_loss(model, data_loader):
    curr_loss, num_examples = 0., 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 784).to(device)
            targets = targets.to(device)
            logits, probas = model.forward(features)
            loss = F.cross_entropy(logits, torch.squeeze(targets), reduction='sum')
            num_examples += targets.size(0)
            curr_loss += loss

        curr_loss = curr_loss / num_examples
        return curr_loss

    
def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 784).to(device)
            targets = targets.to(device)
            targets = torch.squeeze(targets)
            logits, probas = model.forward(features)
            predicted_labels = torch.argmax(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float()/num_examples * 100

In [0]:
trainfoldx = torch.tensor(x_train, dtype=torch.float32).to(device)
trainfoldy = torch.tensor(y_train, dtype=torch.long).to(device)
x_val_fold = torch.tensor(x_test, dtype=torch.float32).to(device)
y_val_fold = torch.tensor(y_test[:,np.newaxis], dtype=torch.long).to(device)

train = torch.utils.data.TensorDataset(trainfoldx,trainfoldy)
valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)

train_loader = torch.utils.data.DataLoader(train , batch_size=batch_size,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

In [0]:
mbatch_cost = []
start_time = time.time()
minibatch_cost = []
epoch_cost = []  
torch.manual_seed(seed)
model = CNN()                   ### Change here if your model name is changed ###
model.to(device)                
optimizer = torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=1e-8)  ### Change here for different optimizer ###

for epoch in range(n_epochs):
    start = time.time()
    model.train()
    grand_loss = 0
    for  batch_idx, (x_batch, y_batch) in enumerate(train_loader):
        logits, probas = model(x_batch)
        loss = F.cross_entropy(logits, y_batch)

        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        mbatch_cost.append(loss.item())
        grand_loss += loss.item()/len(train_loader)
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Loss: %.4f' 
                %(epoch+1, n_epochs, batch_idx, len(train_loader), loss.item()))

                  #evaluation          
    model.eval()
    cost = compute_epoch_loss(model, train_loader)
    cost_test = compute_epoch_loss(model, valid_loader)
    epoch_cost.append(cost)

    train_accuracy = compute_accuracy(model, train_loader)
    valid_accuracy = compute_accuracy(model, valid_loader)

    print('Epoch: %03d/%03d Train Cost: %.4f Test Cost: %.4f' % (
            epoch+1, n_epochs, cost, cost_test))
    print('Train Accuracy: %.3f | Test Accuracy: %.3f' % (train_accuracy, valid_accuracy))
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

In [0]:
### Plot minibatch v.s. loss ###
plt.plot(mbatch_cost)
plt.show()

In [0]:
### Get prediction and model judgement metrics ###
logits, predicted_prob = model(x_val_fold)
fpr, tpr, thresholds = metrics.roc_curve(y_test, np.array(predicted_prob.detach().cpu()))
plt.plot(fpr, tpr, 'b')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('TP')
plt.xlabel('FP')
plt.show()
AUC = metrics.auc(fpr, tpr)
print('AUC=',AUC)
from sklearn.metrics import confusion_matrix
pred = torch.argmax(predicted_prob, 1)
con=confusion_matrix(y_test,pred)
accuracy=(con[0,0]+con[1,1])/(con[0,1]+con[1,0]+con[0,0]+con[1,1])
precision=con[0,0]/(con[0,0]+con[1,0])
recall=con[0,0]/(con[0,0]+con[0,1])
print('accuracy=',accuracy)
print('error=',1-accuracy)
print('precision=',precision)
print('recall=',recall)